In [ ]:
pip install huggingface_hub transformers

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", torch_dtype=torch.bfloat16, device_map="auto")

### Parsing the test descriptions from the files

In [1]:
import os
import importlib.util

tests_data = []

for i in range(1, 11):
    folder_path = f'tests/problem-{i}'
    description_path = os.path.join(folder_path, 'description.txt')
    test_cases_path = os.path.join(folder_path, 'test.py')

    # Read description
    with open(description_path, 'r') as f:
        description = f.read()

    # Import the test_cases array from test.py
    spec = importlib.util.spec_from_file_location("test_module", test_cases_path)
    test_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(test_module)

    test_cases = getattr(test_module, 'test_cases', [])

    tests_data.append({'desc': description, 'tests': test_cases})

# Verify first few
for i in range(min(3, len(tests_data))):
    print(f"Problem {i+1}:")
    print(f"  Description Length: {len(tests_data[i]['desc'])}")
    print(f"  Number of test cases: {len(tests_data[i]['tests'])}")
    if tests_data[i]['tests']:
        print(f"  First test case example: {tests_data[i]['tests'][0]}")
    print("-" * 20)


Problem 1:
  Description Length: 1066
  Number of test cases: 36
  First test case example: {'input': '3+2+1', 'output': '1+2+3'}
--------------------
Problem 2:
  Description Length: 417
  Number of test cases: 20
  First test case example: {'input': 1, 'output': 1}
--------------------
Problem 3:
  Description Length: 667
  Number of test cases: 20
  First test case example: {'input': [[5, 4], [6, 4], [6, 7], [2, 3]], 'output': 3}
--------------------


In [2]:
output_array = []
prompts = []

## Using Stepwise Chain of thought

### Preparing description prompts for the LLMs

In [3]:
for test in tests_data:
    prompt = f'''
    You are an expert programmer. Solve the following coding problem **step by step**, explaining your reasoning as you go. Break down your solution into clear steps before writing the final code.
    Description:
    {test['desc']}

Instructions:
1. First, restate the problem in your own words.
2. Identify the inputs and outputs.
3. Outline a step-by-step plan to solve the problem.
4. Explain your reasoning at each step.
5. Write clean, correct code implementing your plan. Write a function name Solve.

Answer in a stepwise manner and clearly label each step.
'''

    prompts.append(prompt)

In [4]:
#convert prompts to json file
import json
with open('prompts.json', 'w') as f:
    json.dump(prompts, f, indent=4)

### Using the Zephyr model from hugging face

In [ ]:
for test in tests_data:
  counter = 0
  prompt = f'''
Problem:
{test['desc']}

Instructions:
1. First, restate the problem in your own words.
2. Identify the inputs and outputs.
3. Outline a step-by-step plan to solve the problem.
4. Explain your reasoning at each step.
5. Write clean, correct code implementing your plan. Write a function name Solve.

Answer in a stepwise manner and clearly label each step.
'''
  messages = [
    {
        "role": "system",
        "content": "You are an expert programmer. Solve the following coding problem **step by step**, explaining your reasoning as you go. Break down your solution into clear steps before writing the final code.",
    },
    {"role": "user", "content": prompt},
]
  prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
  outputs = pipe(prompt, max_new_tokens=1024, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
  output_array.append((outputs[0]["generated_text"]))

  print(counter)
  counter += 1




In [ ]:
with open('zephyr_outputs.txt', 'w') as f:
    for output in output_array:
        f.write(output + "\n\n---\n\n")

### Using the Gemini model from google

In [6]:
!pip install -q -U google-genai

You should consider upgrading via the 'C:\Users\Kenne\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
%env GEMINI_API_KEY=

In [10]:
from google import genai

client = genai.Client()

output_array = []
for prompt in prompts:
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt,
    )
    output_array.append(response.text)

with open('gemini_outputs.txt', 'w') as f:
    for output in output_array:
        f.write(output + "\n\n---\n\n")